In [1]:
#hide
!nvidia-smi

Thu Apr 29 14:53:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        On   | 00000000:00:05.0 Off |                  Off |
| 26%   25C    P8     6W / 180W |      1MiB / 16278MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#all_slow

In [3]:
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq fastcore sentencepiece
    !pip install -Uqq --no-deps fastai
    !pip install -Uqq transformers datasets wandb 

In [2]:
from transformers import AutoModelForMaskedLM, DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from datasets import load_dataset, concatenate_datasets, load_metric

from fastai.text.all import *
from fasthugs.learner import TransLearner
from fasthugs.data import *

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


## Setup

In [4]:
model_name = 'distilroberta-base'
# data
max_length = 128
bs = 16
val_bs = bs*4
# training
lr = 3e-5

## Data preprocessing

In this example notebook we use HuggingFace datasets for preprocessing (as show in example [here]().

In [5]:
ds_name = 'imdb'

In [6]:
dataset = load_dataset(ds_name)

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/4ea52f2e58a08dbc12c2bd52d0d92b30b88c00230b4522801b3636782f625c5b)


In [7]:
# train_ds = train_ds.select(range(100))
# valid_ds = valid_ds.select(range(100))

In [7]:
dataset.column_names

{'train': ['label', 'text'],
 'test': ['label', 'text'],
 'unsupervised': ['label', 'text']}

In [8]:
dataset['unsupervised'][2]

{'label': -1,
 'text': "Everybody has seen 'Back To The Future,' right? Whether you LIKE that movie or not, you've seen an example of how to make a time-travel movie work. A torn-up poster for 'Back To The Future' shows up in this movie, representing, perhaps unintentionally, what the makers of 'Tangents' (aka 'Time Chasers') did to the time-travel formula. Then again, the movie claims to have been made in 1994, but it looks -- and sounds -- like it was produced at least ten years earlier, so maybe they achieved time-travel after all.<br /><br />Start with an intensely unappealing leading man. I mean, what woman doesn't love gangly, whiny, lantern-jawed, butt-chinned, mullet-men with Coke-bottle glasses? Oh, none of you? Prepare to tough it out, ladies, cuz that's what this movie gives you.<br /><br />Second, add a leading lady who -- while not entirely unattractive -- represents many '80s clichés: big hair, too much makeup, two different plaids, shoulder pads, acid-washed mom-jeans, e

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
def tokenize(batch):
    return tokenizer(batch['text'],return_attention_mask=True, return_special_tokens_mask=True)

In [11]:
dataset = dataset.map(tokenize, batched=True, batch_size=100, remove_columns=dataset['train'].column_names, num_proc=4)

Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (897 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (950 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (803 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (1289 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (540 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (699 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (525 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (523 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (723 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1068 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (565 > 512). Running this sequence through the model will result in indexing errors


In [13]:
block_size = max_len

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [14]:
lm_dataset = dataset.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [15]:
lm_dataset = concatenate_datasets([lm_dataset['train'], lm_dataset['unsupervised'], lm_dataset['test']])

## Training

In [17]:
import random

In [19]:
# N = len(lm_dataset)
N = 1000
idx = list(range(N))
random.shuffle(idx)

In [20]:
split = int(N*0.9)
train_idx = idx[:split]
valid_idx = idx[split:]

In [21]:
model = AutoModelForMaskedLM.from_pretrained(model_name)

In [22]:
ds = lm_dataset.select(list(range(N)))

In [26]:
dsets = Datasets(ds, [[noop],], splits=IndexSplitter(valid_idx)(ds))

In [47]:
def untuple(l):
    return [e[0] for e in l]

In [29]:
def to_tuple(x):
    return (x, )

In [33]:
class TransformersLMBlock(TransformBlock):
    "A `TransformBlock` for texts using pretrained tokenizers from Huggingface"
    #@delegates(TokBatchTransform)
    def __init__(self, pretrained_model_name=None, tokenizer_cls=AutoTokenizer, 
                 config=None, tokenizer=None, is_lm=True, **kwargs):
#         before_batch_tfm = TokBatchTransform(pretrained_model_name=pretrained_model_name, tokenizer_cls=tokenizer_cls, 
#                  config=config, tokenizer=tokenizer, return_special_tokens_mask=True, is_lm=True, **kwargs)
        create_batch = compose(DataCollatorForLanguageModeling(tokenizer), to_tuple)
        return super().__init__(dl_type=TfmdDL,
                                dls_kwargs={
                                    #'before_batch': before_batch_tfm,
                                    'create_batch': create_batch},
                                # batch_tfms=Undict()
                               )

In [52]:
dblock = DataBlock(blocks=[TransformersLMBlock(tokenizer=tokenizer)], 
                   get_x=noop,
                   splitter=IndexSplitter(valid_idx))

The labels are constructed by `DataCollatorForLanguageModeling` and the loss computed by the model is used for training.

In [71]:
learn = TransLearner(dls, model, loss_func=noop, splitter=default_splitter, metrics=perplexity)

In [ ]:
learn.fit(5, 3e-5)

epoch,train_loss,valid_loss,perplexity,time
0,2.448122,2.199647,9.021828,04:29
1,2.379829,2.337306,10.353308,04:59
